In [4]:
import os

from configs.config import ConfigPath
from data_collection.article_fetcher import PubMedArticleFetcher
from data_collection.reader import MetadataReader

In [5]:
# read pmids
external_metadata_file_path = os.path.join(ConfigPath.EXTERNAL_DATA_DIR, "pqa_labeled.json")
metadata_reader = MetadataReader(file_path=external_metadata_file_path)

2025-01-23 20:40:54,937 [DEBUG] reader - External metadata file loaded successfully!


In [7]:
# download articles

fetcher = PubMedArticleFetcher(data_path=ConfigPath.RAW_DATA_DIR)

fetcher.fetch_articles(pmids=metadata_reader.pmids)

2025-01-23 20:42:03,827 [INFO] article_fetcher - Fetching articles for PMIDs: 21645374, 16418930, 9488747, 17208539, 10808977, 23831910, 26037986, 26852225, 17113061, 10966337, 25432938, 18847643, 18239988, 25957366, 24866606, 26578404, 11729377, 17096624, 22694248, 22990761, 19394934, 11481599, 21669959, 23806388, 17919952, 10966943, 23690198, 17940352, 20537205, 28707539, 7482275, 24183388, 9645785, 26298839, 24153338, 18534072, 8847047, 10575390, 20084845, 15703931, 18269157, 25489696, 14599616, 22537902, 19054501, 16432652, 19504993, 20571467, 24237112, 21402341, 20082356, 26606599, 11340218, 25481573, 25277731, 25475395, 23177368, 17179167, 14612308, 27491658, 19822586, 27643961, 23539689, 22453060, 22227642, 12380309, 22186742, 22188074, 27989969, 18607272, 18235194, 18049437, 15597845, 24996865, 23361217, 17489316, 14518645, 20337202, 26449554, 26215326, 29112560, 24315783, 9381529, 8985020, 26708803, 22427593, 11570976, 19757704, 27592038, 27044366, 9792366, 17562682, 15800018,